## 2RPFS Problem (TWCT objective) - Tables and Graphs

Before running this, notebook, please run notebook 0.1.

In [1]:
import pandas as pd
import numpy as np
import os, fnmatch
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
import glob
import seaborn as sns
import gzip
import matplotlib.style as style
from matplotlib.path import Path
from matplotlib.patches import BoxStyle

%matplotlib inline

In [2]:
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

In [3]:
linestyle_tuple = [
     ('dotted',                (0, (1, 1))),
     ('dashed',                (0, (5, 5))),
     ('densely dashed',        (0, (5, 1))),
     ('dashdotdotted',         (0, (3, 5, 1, 5, 1, 5))),
     ('densely dashdotdotted', (0, (3, 1, 1, 1, 1, 1))),

     ('dashdotted',            (0, (3, 5, 1, 5))),
     ('densely dashdotted',    (0, (3, 1, 1, 1))),
     
     ('loosely dashed',        (0, (5, 10))),
     ('loosely dashdotted',    (0, (3, 10, 1, 10))),
     

     ('loosely dashdotdotted', (0, (3, 10, 1, 10, 1, 10))),
     ('densely dotted',        (0, (1, 1))),
     ('loosely dotted',        (0, (1, 10)))]

In [4]:
# https://stackoverflow.com/questions/51483901/is-there-a-way-to-extend-the-solid-color-background-to-the-full-width-of-the-pag
class ExtendedTextBox(BoxStyle._Base):
    """
    An Extended Text Box that expands to the axes limits 
                        if set in the middle of the axes
    """

    def __init__(self, pad=0.3, width=500.):
        """
        width: 
            width of the textbox. 
            Use `ax.get_window_extent().width` 
                   to get the width of the axes.
        pad: 
            amount of padding (in vertical direction only)
        """
        self.width=width
        self.pad = pad
        super(ExtendedTextBox, self).__init__()

    def transmute(self, x0, y0, width, height, mutation_size):
        """
        x0 and y0 are the lower left corner of original text box
        They are set automatically by matplotlib
        """
        # padding
        pad = mutation_size * self.pad

        # we add the padding only to the box height
        height = height + 2.*pad
        # boundary of the padded box
        y0 = y0 - pad
        y1 = y0 + height
        _x0 = x0
        x0 = _x0 +width /2. - self.width/2.
        x1 = _x0 +width /2. + self.width/2.

        cp = [(x0, y0),
              (x1, y0), (x1, y1), (x0, y1),
              (x0, y0)]

        com = [Path.MOVETO,
               Path.LINETO, Path.LINETO, Path.LINETO,
               Path.CLOSEPOLY]

        path = Path(cp, com)

        return path

### List files in the result folder 

In [5]:
resultfolder = os.path.join(os.getcwd(), 'results', 'consolidated')
rpfs_file = os.path.join(resultfolder, 'RPFS_TWCT_all_results.pkl.gz')

### Create the output folder 

In [6]:
outputfolder = os.path.join(os.getcwd(), 'results', 'consolidated')
outputfolder_graph = os.path.join(os.getcwd(), 'results', 'consolidated', 'graphs')
outputfolder_table = os.path.join(os.getcwd(), 'results', 'consolidated', 'tables')
if not os.path.exists(outputfolder_graph):
    os.makedirs(outputfolder_graph)
if not os.path.exists(outputfolder_table):
    os.makedirs(outputfolder_table)
#print('Saving files on folder: ' + outputfolder)

### Process consolidated CSV result files

In [7]:
df_rpfs = pd.read_pickle(rpfs_file)  # Robust PFSP Budget solutions only
df_rpfs.drop(columns=['executionId'], inplace=True)
df_rpfs = df_rpfs.reset_index()

**Robust dataframe: calculating new fields.**

In [8]:
df_rpfs['optimal'] = df_rpfs['is_optimal'] & df_rpfs['validated'] & (df_rpfs['gap'] <= 1e-8)
df_rpfs['time_limit'] = 7200.0
df_rpfs['time_limit_2'] = 7200.0 * 2
df_rpfs['mp_total_time'] = (df_rpfs['n'] < 15).astype(int) * np.minimum(df_rpfs['mp_total_time'], df_rpfs['time_limit']) + (df_rpfs['n'] >= 15).astype(int) * np.minimum(df_rpfs['mp_total_time'], df_rpfs['time_limit_2'])
df_rpfs['time'] = df_rpfs['mp_total_time'] + df_rpfs['sp_total_time']
df_rpfs['gap'] = df_rpfs['gap'] * 100.0
df_rpfs['RobCost_worstcase'] = df_rpfs['wct_validation']
df_rpfs = df_rpfs.rename(columns={"budget_Gamma": "RobCost_Gamma"})

In [9]:
df_rpfs.tail(4)

,model,n,m,alpha,seq,RobCost_Gamma,instance_type,ub_name,instance_name,wct,...,wct_validation,cut_count,Gamma%,n_str,alpha_str,optimal,time_limit,time_limit_2,time,RobCost_worstcase
18415,wilson,15,5,50.0,01,52.5,ying,mip_separation,RB0151001_15_5_R50_wct_inputs.txt,253013.47,...,253013.47,2,70.0,015,50.0,False,7200.0,14400.0,14399.66,253013.47
18416,wilson,15,5,50.0,01,60.0,ying,mip_separation,RB0151001_15_5_R50_wct_inputs.txt,250828.37,...,250828.37,3,80.0,015,50.0,False,7200.0,14400.0,14399.42,250828.37
18417,wilson,15,5,50.0,01,67.5,ying,mip_separation,RB0151001_15_5_R50_wct_inputs.txt,254859.41,...,254859.41,2,90.0,015,50.0,False,7200.0,14400.0,14399.18,254859.41
18418,wilson,15,5,50.0,01,75.0,ying,mip_separation,RB0151001_15_5_R50_wct_inputs.txt,250828.37,...,250828.37,1,100.0,015,50.0,True,7200.0,14400.0,2812.44,250828.37


In [10]:
df_rpfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18419 entries, 0 to 18418
Data columns (total 33 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   model                  18419 non-null  object 
 1   n                      18419 non-null  int64  
 2   m                      18419 non-null  int64  
 3   alpha                  18391 non-null  float64
 4   seq                    18419 non-null  object 
 5   RobCost_Gamma          18419 non-null  float64
 6   instance_type          18419 non-null  object 
 7   ub_name                18419 non-null  object 
 8   instance_name          18419 non-null  object 
 9   wct                    18414 non-null  float64
 10  permutation            18419 non-null  object 
 11  time_spent             18391 non-null  float64
 12  time_to_best_sol       18419 non-null  float64
 13  mp_total_time          18419 non-null  float64
 14  sp_total_time          18419 non-null  float64
 15  it

### Checking the Robust PFSP Budget solutions dataframe

In [11]:
df_rpfs.head(2)

,model,n,m,alpha,seq,RobCost_Gamma,instance_type,ub_name,instance_name,wct,...,wct_validation,cut_count,Gamma%,n_str,alpha_str,optimal,time_limit,time_limit_2,time,RobCost_worstcase
0,hybrid,10,2,0.0,01,2.0,ying,mip_separation,RB0101001_10_2_R400_wct_inputs.txt,116043.57,...,116043.57,5,10.0,010,0.0,True,7200.0,14400.0,405.41,116043.57
1,hybrid,10,2,0.0,01,4.0,ying,mip_separation,RB0101001_10_2_R400_wct_inputs.txt,153238.39,...,153238.39,8,20.0,010,0.0,True,7200.0,14400.0,580.63,153238.39


# Tables

Obtain list of C&CG models, instance types

In [12]:
model_list = df_rpfs['model'].unique().tolist()
instance_type_list = df_rpfs['instance_type'].unique().tolist()
print(model_list)
print(instance_type_list)

['hybrid', 'liao-you', 'manne', 'tba', 'ts2', 'ts3', 'wagner-wst2', 'wilson']
['ying']


Add a new column containing the instance size as string

In [13]:
df_temp = df_rpfs
(df_temp['n'].astype(str) + 'x' + df_temp['m'].astype(str)).unique()

array(['10x2', '10x3', '10x4', '10x5', '15x5', '10x10'], dtype=object)

In [14]:
df_rpfs.columns

Index(['model', 'n', 'm', 'alpha', 'seq', 'RobCost_Gamma', 'instance_type',
       'ub_name', 'instance_name', 'wct', 'permutation', 'time_spent',
       'time_to_best_sol', 'mp_total_time', 'sp_total_time', 'iterations',
       'num_visited_solutions', 'num_improvements', 'is_optimal', 'validated',
       'gap', 'lb', 'cost', 'wct_validation', 'cut_count', 'Gamma%', 'n_str',
       'alpha_str', 'optimal', 'time_limit', 'time_limit_2', 'time',
       'RobCost_worstcase'],
      dtype='object')

In [15]:
df_temp = df_rpfs
df_temp['instance_size'] = df_temp['n'].astype(str) + 'x' + df_temp['m'].astype(str)
df_rpfs = df_temp.set_index(['model', 'n', 'm', 'alpha', 'seq', 'RobCost_Gamma', 'instance_type'])
df_rpfs

ub_name  \
model  n  m alpha seq RobCost_Gamma instance_type                   
hybrid 10 2 0.0   01  2.0           ying           mip_separation   
                      4.0           ying           mip_separation   
                      6.0           ying           mip_separation   
                      8.0           ying           mip_separation   
                      10.0          ying           mip_separation   
...                                                           ...   
wilson 15 5 50.0  01  45.0          ying           mip_separation   
                      52.5          ying           mip_separation   
                      60.0          ying           mip_separation   
                      67.5          ying           mip_separation   
                      75.0          ying           mip_separation   

                                                                        instance_name  \
model  n  m alpha seq RobCost_Gamma instance_type                                       
hybrid 10 2 0.0   01  2.0           ying           RB0101001_10_2_R400_wct_inputs.txt   
                      4.0           ying           RB0101001_10_2_R400_wct_inputs.txt   
                      6.0           ying           RB0101001_10_2_R400_wct_inputs.txt   
                      8.0           ying           RB0101001_10_2_R400_wct_inputs.txt   
                      10.0          ying           RB0101001_10_2_R400_wct_inputs.txt   
...                                                                               ...   
wilson 15 5 50.0  01  45.0          ying            RB0151001_15_5_R50_wct_inputs.txt   
                      52.5          ying            RB0151001_15_5_R50_wct_inputs.txt   
                      60.0          ying            RB0151001_15_5_R50_wct_inputs.txt   
                      67.5          ying            RB0151001_15_5_R50_wct_inputs.txt   
                      75.0          ying            RB0151001_15_5_R50_wct_inputs.txt   

                                                         wct  \
model  n  m alpha seq RobCost_Gamma instance_type              
hybrid 10 2 0.0   01  2.0           ying           116043.57   
                      4.0           ying           153238.39   
                      6.0           ying           175669.94   
                      8.0           ying           187399.41   
                      10.0          ying           194179.33   
...                                                      ...   
wilson 15 5 50.0  01  45.0          ying           255028.00   
                      52.5          ying           253013.47   
                      60.0          ying           250828.37   
                      67.5          ying           254859.41   
                      75.0          ying           250828.37   

                                                                           permutation  \
model  n  m alpha seq RobCost_Gamma instance_type                                        
hybrid 10 2 0.0   01  2.0           ying                          1 3 7 6 8 10 9 4 2 5   
                      4.0           ying                          10 7 8 1 3 6 9 4 2 5   
                      6.0           ying                          10 7 8 6 1 3 9 4 2 5   
                      8.0           ying                          10 7 6 8 1 9 3 4 2 5   
                      10.0          ying                          7 6 10 8 1 9 3 4 2 5   
...                                                                                ...   
wilson 15 5 50.0  01  45.0          ying           10 1 11 2 4 8 15 12 13 3 7 14 9 6 5   
                      52.5          ying           11 8 1 4 10 2 15 12 13 3 7 9 14 6 5   
                      60.0          ying           11 10 1 4 8 2 15 12 13 3 7 9 14 6 5   
                      67.5          ying           11 4 8 1 10 2 15 12 13 3 7 9 14 6 5   
                      75.0          ying           11 10 1 4 8 2 15 12 13 3 7 9 14 6 5   

Treating errors in the `gap` column

In [16]:
df_rpfs['gap'].describe()

count    1.825300e+04
mean     3.650261e-01
std      2.577871e+00
min     -1.193777e+00
25%      0.000000e+00
50%      1.871615e-14
75%      3.663496e-13
max      1.143266e+02
Name: gap, dtype: float64

In [17]:
df_rpfs['gap'] = df_rpfs['gap'].apply(lambda x: np.maximum(x, 0.0))

In [18]:
df_rpfs['gap'].describe()

count    1.825300e+04
mean     3.654778e-01
std      2.577769e+00
min      0.000000e+00
25%      0.000000e+00
50%      1.871615e-14
75%      3.663496e-13
max      1.143266e+02
Name: gap, dtype: float64

## Table 2. Performance given all instances 

Model-wise Robust PFSP C&CG performance comparison, given all instances.

* % Best Performance is the percentage of instances solved to optimality where the model achieved shorter execution time, when compared to the other models; 

* % Solved contains the percentage of instances solved within the time limit; 

* % Solved < n x m > represents the percentage of solved instances of size n x m; 

* Avg. % Gap is the average percentage gap of solutions from instances not solved to optimality; 

* Median time is the median execution time, in seconds; 

* Median iterations is the median of the number of iterations performed.

In [19]:
df_model = df_rpfs.reset_index()
df_model = df_model[df_model['model'] == 'hybrid']
df_model = df_model[df_model['optimal'] == True]    
df_model = df_model.set_index(['n', 'm', 'alpha', 'seq', 'RobCost_Gamma', 'instance_type'])

df_others = df_rpfs.reset_index()
df_others = df_others[df_others['model'] != 'hybrid']
df_others = df_others[df_others['optimal'] == True] 
group_columns = ['n', 'm', 'alpha', 'seq', 'RobCost_Gamma', 'instance_type']
df_best_performance = df_others[group_columns + ['time']].groupby(by=group_columns).min()['time']
df_best_performance = df_best_performance.to_frame()
df_best_performance

time
n  m alpha seq RobCost_Gamma instance_type          
10 2 0.0   01  0.4           ying             123.94
               0.8           ying             363.21
               1.2           ying             908.01
               1.6           ying            1712.98
               2.0           ying              27.52
...                                              ...
15 5 50.0  01  75.0          ying            2669.40
           03  52.5          ying            6936.22
               60.0          ying            8229.22
               67.5          ying           11388.60
               75.0          ying            2357.20

[2871 rows x 1 columns]

In [20]:
df_compare = df_best_performance.join(df_model, how='inner', 
                                                     on=['n', 'm', 'alpha', 'seq', 'RobCost_Gamma', 'instance_type'],
                                                     lsuffix='_best')
df_compare['time_wins'] = (df_compare['time'] < df_compare['time_best']).astype(int)
df_compare

time_best   model         ub_name  \
n  m alpha seq RobCost_Gamma instance_type                                      
10 2 0.0   01  2.0           ying               27.52  hybrid  mip_separation   
               4.0           ying               32.48  hybrid  mip_separation   
               6.0           ying              138.30  hybrid  mip_separation   
               8.0           ying              213.82  hybrid  mip_separation   
               10.0          ying              148.78  hybrid  mip_separation   
...                                               ...     ...             ...   
   5 50.0  02  50.0          ying               16.15  hybrid  mip_separation   
15 5 0.0   01  75.0          ying            11735.11  hybrid  mip_separation   
     10.0  01  60.0          ying            13537.43  hybrid  mip_separation   
               67.5          ying            11403.22  hybrid  mip_separation   
               75.0          ying              931.15  hybrid  mip_separation   

                                                                 instance_name  \
n  m alpha seq RobCost_Gamma instance_type                                       
10 2 0.0   01  2.0           ying           RB0101001_10_2_R400_wct_inputs.txt   
               4.0           ying           RB0101001_10_2_R400_wct_inputs.txt   
               6.0           ying           RB0101001_10_2_R400_wct_inputs.txt   
               8.0           ying           RB0101001_10_2_R400_wct_inputs.txt   
               10.0          ying           RB0101001_10_2_R400_wct_inputs.txt   
...                                                                        ...   
   5 50.0  02  50.0          ying            RB0101002_10_5_R50_wct_inputs.txt   
15 5 0.0   01  75.0          ying           RB0151001_15_5_R100_wct_inputs.txt   
     10.0  01  60.0          ying             RB0151001_15_5_10_wct_inputs.txt   
               67.5          ying             RB0151001_15_5_10_wct_inputs.txt   
               75.0          ying             RB0151001_15_5_10_wct_inputs.txt   

                                                  wct  \
n  m alpha seq RobCost_Gamma instance_type              
10 2 0.0   01  2.0           ying           116043.57   
               4.0           ying           153238.39   
               6.0           ying           175669.94   
               8.0           ying           187399.41   
               10.0          ying           194179.33   
...                                               ...   
   5 50.0  02  50.0          ying           109253.66   
15 5 0.0   01  75.0          ying           307101.39   
     10.0  01  60.0          ying           222379.30   
               67.5          ying           222379.30   
               75.0          ying           222379.30   

                                                                    permutation  \
n  m alpha seq RobCost_Gamma instance_type                                        
10 2 0.0   01  2.0           ying                          1 3 7 6 8 10 9 4 2 5   
               4.0           ying                          10 7 8 1 3 6 9 4 2 5   
               6.0           ying                          10 7 8 6 1 3 9 4 2 5   
               8.0           ying                          10 7 6 8 1 9 3 4 2 5   
               10.0          ying                          7 6 10 8 1 9 3 4 2 5   
...                                                                         ...   
   5 50.0  02  50.0          ying                          10 9 3 2 1 7 6 8 5 4   
15 5 0.0   01  75.0          ying           11 2 4 8 1 15 10 12 7 9 13 3 14 6 5   
     10.0  01  60.0          ying           10 1 11 2 4 8 15 12 13 3 7 14 9 6 5   
               67.5          ying           10 1 11 2 4 8 15 12 13 3 7 14 9 6 5   
               75.0          ying           10 1 11 2 4 8 15 12 13 3 7 14 9 6 5   

                                            time_spent  time_to_best_sol  \
n  m alpha seq RobCost_Ga

In [21]:
def calculate_perc_best_performance(df, model):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    df_model = df_model[df_model['optimal'] == True]    
    df_model = df_model.set_index(['n', 'm', 'alpha', 'seq', 'RobCost_Gamma', 'instance_type'])
    if len(df_model.index) == 0:
        return np.nan
    
    df_others = df.reset_index()
    df_others = df_others[df_others['model'] != model]
    df_others = df_others[df_others['optimal'] == True] 
    group_columns = ['n', 'm', 'alpha', 'seq', 'RobCost_Gamma', 'instance_type']
    df_best_performance = df_others[group_columns + ['time']].groupby(by=group_columns).min()['time']
    df_best_performance = df_best_performance.to_frame()
    if len(df_best_performance.index) == 0:
        return np.nan
    
    df_compare = df_best_performance.join(df_model, how='inner', 
                                                     on=group_columns,
                                                     lsuffix='_best')
    df_compare['time_wins'] = (df_compare['time'] < df_compare['time_best']).astype(int)
    return np.round(100.0 * df_compare['time_wins'].sum() / len(df_compare.index), 2)

In [22]:
def calculate_perc_solved(df, model, instance_type = None, instance_size = None):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    df_ = df_model
    if instance_type is not None:
        df_ = df_[df_['instance_type'] == instance_type]
    if instance_size is not None:
        df_ = df_[df_['instance_size'] == instance_size]
    if len(df_.index) > 0:
        return np.round(100.0 * len(df_[(df_['optimal'] == True)].index) / len(df_.index), 2)
    else:
        return np.nan

In [23]:
# Avg. % Gap is the average percentage gap of solutions from instances not solved to optimality
def calculate_avg_perc_gap(df, model):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    df_model = df_model[df_model['optimal'] == False]
    return np.round(df_model['gap'].mean(), 2)

In [24]:
def calculate_median_time(df, model, time_col_name):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    return np.round(df_model[time_col_name].median(), 2)

In [25]:
def calculate_avg_time(df, model, time_col_name):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    return np.round(df_model[time_col_name].mean(), 2)

In [26]:
def calculate_std_time(df, model, time_col_name):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    return np.round(df_model[time_col_name].std(), 2)

In [27]:
def calculate_median_iterations(df, model):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    return np.round(df_model['iterations'].median(), 2)

In [28]:
def calculate_avg_iterations(df, model):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    return np.round(df_model['iterations'].mean(), 2)

In [29]:
def calculate_std_iterations(df, model):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    return np.round(df_model['iterations'].std(), 2)

In [30]:
model_stats = dict()
for model in model_list:
    model_stats[model] = dict()
    model_stats[model]['% Best Performance'] = calculate_perc_best_performance(df_rpfs, model)
    model_stats[model]['% Solved'] = calculate_perc_solved(df_rpfs, model)  # given all instances
    for instance_type in instance_type_list:  # group by instance type and size
        df_itype = df_rpfs.reset_index()
        df_itype = df_itype[(df_itype['instance_type'] == instance_type)]
        instance_size_list = df_itype['instance_size'].unique().tolist()
        for instance_size in instance_size_list:
            model_stats[model]['% Solved '+instance_type+' '+ instance_size] = calculate_perc_solved(df_rpfs, model, instance_type, instance_size)
    model_stats[model]['Avg. % gap'] = calculate_avg_perc_gap(df_rpfs, model)
    model_stats[model]['Median time'] = calculate_median_time(df_rpfs, model, 'time')
    model_stats[model]['Median time MP'] = calculate_median_time(df_rpfs, model, 'mp_total_time')
    model_stats[model]['Median time SP'] = calculate_median_time(df_rpfs, model, 'sp_total_time')
    model_stats[model]['Median iterations'] = calculate_median_iterations(df_rpfs, model)

In [31]:
model_stats

{'hybrid': {'% Best Performance': 26.84,
  '% Solved': 96.83,
  '% Solved ying 10x2': 99.33,
  '% Solved ying 10x3': 98.06,
  '% Solved ying 10x4': 88.33,
  '% Solved ying 10x5': 98.0,
  '% Solved ying 15x5': 70.0,
  '% Solved ying 10x10': nan,
  'Avg. % gap': 0.46,
  'Median time': 51.4,
  'Median time MP': 49.48,
  'Median time SP': 1.06,
  'Median iterations': 4.0},
 'liao-you': {'% Best Performance': 16.64,
  '% Solved': 92.8,
  '% Solved ying 10x2': 100.0,
  '% Solved ying 10x3': 99.67,
  '% Solved ying 10x4': 96.49,
  '% Solved ying 10x5': 94.32,
  '% Solved ying 15x5': 12.86,
  '% Solved ying 10x10': 0.0,
  'Avg. % gap': 3.25,
  'Median time': 145.11,
  'Median time MP': 97.51,
  'Median time SP': 10.9,
  'Median iterations': 4.0},
 'manne': {'% Best Performance': 53.47,
  '% Solved': 93.47,
  '% Solved ying 10x2': 100.0,
  '% Solved ying 10x3': 99.67,
  '% Solved ying 10x4': 96.33,
  '% Solved ying 10x5': 97.16,
  '% Solved ying 15x5': 14.79,
  '% Solved ying 10x10': 0.0,
  'Av

In [32]:
model_stats_df = pd.DataFrame.from_dict(model_stats)
model_stats_df

,hybrid,liao-you,manne,tba,ts2,ts3,wagner-wst2,wilson
% Best Performance,26.84,16.64,53.47,0.10,7.17,2.02,0.00,11.93
% Solved,96.83,92.80,93.47,81.92,89.20,88.47,81.17,86.28
% Solved ying 10x2,99.33,100.00,100.00,99.00,100.00,99.83,97.33,100.00
% Solved ying 10x3,98.06,99.67,99.67,88.48,93.99,91.00,86.81,92.49
% Solved ying 10x4,88.33,96.49,96.33,75.96,86.64,81.50,74.12,85.83
% Solved ying 10x5,98.00,94.32,97.16,64.83,80.13,82.14,66.94,73.79
% Solved ying 15x5,70.00,12.86,14.79,NaN,8.00,NaN,NaN,6.25
% Solved ying 10x10,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Avg. % gap,0.46,3.25,6.00,3.07,4.07,2.62,2.92,3.97
Median time,51.40,145.11,97.44,813.34,355.66,286.41,987.17,240.79


In [33]:
# Save output table as HTML
pd.set_option('colheader_justify', 'center')   # FOR TABLE <th>

html_string = '''
<html>
  <head><title>HTML Pandas Dataframe with CSS</title></head>
  <link rel="stylesheet" type="text/css" href="df_style.css"/>
  <body>
    {table}
  </body>
</html>.
'''

# OUTPUT AN HTML FILE
with open(os.path.join(outputfolder_table, 'model_stats.html'), 'w') as f:
    f.write(html_string.format(table=model_stats_df.to_html(classes='mystyle')))

## Table 3. Performance per instance group and model

Model-wise Robust PFSP C&CG performance comparison, per instance group.

* % Best Performance is the percentage of instances solved to optimality where the model achieved shorter execution time, when compared to the other models; 

* % Solved contains the percentage of instances solved within the time limit; 

* Avg. % Gap is the average percentage gap of solutions from instances not solved to optimality; 

* Avg. time and Std. dev. of time are the mean and standard deviation in solution time (s), respectively;

* Avg. iterations and Std. dev. of iterations are the mean and standard deviation of the number of iterations performed.

In [34]:
per_instance_stats = dict()
for instance_type in instance_type_list:  # group by instance type and size
    df_itype = df_rpfs.reset_index()
    df_itype = df_itype[(df_itype['instance_type'] == instance_type)]
    instance_size_list = df_itype['instance_size'].unique().tolist()
    for instance_size in instance_size_list:
        df_instance = df_itype[df_itype['instance_size'] == instance_size]
        for model in model_list:
            per_instance_stats[(instance_type,instance_size,model)] = dict()
            per_instance_stats[(instance_type,instance_size,model)]['% Best Performance'] = calculate_perc_best_performance(df_instance, model)
            per_instance_stats[(instance_type,instance_size,model)]['% Solved'] = calculate_perc_solved(df_rpfs, model, instance_type, instance_size)
            per_instance_stats[(instance_type,instance_size,model)]['Avg. % gap'] = calculate_avg_perc_gap(df_instance, model)
            per_instance_stats[(instance_type,instance_size,model)]['Avg. time'] = calculate_avg_time(df_instance, model, 'time')
            per_instance_stats[(instance_type,instance_size,model)]['Std. dev. of time'] = calculate_std_time(df_instance, model, 'time')
            per_instance_stats[(instance_type,instance_size,model)]['Avg. MP time'] = calculate_avg_time(df_instance, model, 'mp_total_time')
            per_instance_stats[(instance_type,instance_size,model)]['Avg. SP time'] = calculate_avg_time(df_instance, model, 'sp_total_time')
            per_instance_stats[(instance_type,instance_size,model)]['Avg. iterations'] = calculate_avg_iterations(df_instance, model)
            per_instance_stats[(instance_type,instance_size,model)]['Std. dev. of iterations'] = calculate_std_iterations(df_instance, model)

In [35]:
per_instance_stats

{('ying', '10x2', 'hybrid'): {'% Best Performance': 12.25,
  '% Solved': 99.33,
  'Avg. % gap': 0.0,
  'Avg. time': 109.22,
  'Std. dev. of time': 592.47,
  'Avg. MP time': 108.19,
  'Avg. SP time': 1.03,
  'Avg. iterations': 7.83,
  'Std. dev. of iterations': 43.17},
 ('ying', '10x2', 'liao-you'): {'% Best Performance': 8.35,
  '% Solved': 100.0,
  'Avg. % gap': nan,
  'Avg. time': 73.7,
  'Std. dev. of time': 171.29,
  'Avg. MP time': 73.26,
  'Avg. SP time': 0.44,
  'Avg. iterations': 3.94,
  'Std. dev. of iterations': 2.87},
 ('ying', '10x2', 'manne'): {'% Best Performance': 63.94,
  '% Solved': 100.0,
  'Avg. % gap': nan,
  'Avg. time': 31.54,
  'Std. dev. of time': 59.84,
  'Avg. MP time': 30.44,
  'Avg. SP time': 1.1,
  'Avg. iterations': 3.74,
  'Std. dev. of iterations': 2.8},
 ('ying', '10x2', 'tba'): {'% Best Performance': 0.0,
  '% Solved': 99.0,
  'Avg. % gap': 0.54,
  'Avg. time': 404.09,
  'Std. dev. of time': 1054.11,
  'Avg. MP time': 403.55,
  'Avg. SP time': 0.54,
  

In [36]:
# https://stackoverflow.com/questions/57606801/pandas-style-options-to-latex

In [37]:
pd.set_option('display.max_columns', None)
allowed_keys = [(x, y, z) for (x, y, z) in per_instance_stats.keys() if (x == 'ying' and y in ['10x2', '10x3', '10x4'])]
per_instance_stats1 = { your_key: per_instance_stats[your_key] for your_key in allowed_keys }
df_table3a = pd.DataFrame.from_dict(per_instance_stats1)
df_table3a

ying                                             \
                          10x2                                              
                         hybrid liao-you  manne    tba      ts2     ts3     
% Best Performance        12.25    8.35    63.94     0.00    7.18    1.67   
% Solved                  99.33  100.00   100.00    99.00  100.00   99.83   
Avg. % gap                 0.00     NaN      NaN     0.54     NaN    0.61   
Avg. time                109.22   73.70    31.54   404.09  260.94  188.97   
Std. dev. of time        592.47  171.29    59.84  1054.11  691.79  534.15   
Avg. MP time             108.19   73.26    30.44   403.55  260.55  188.50   
Avg. SP time               1.03    0.44     1.10     0.54    0.39    0.46   
Avg. iterations            7.83    3.94     3.74     4.04    3.77    3.94   
Std. dev. of iterations   43.17    2.87     2.80     5.76    2.84    3.54   

                                                                       \
                                              10x3                      
                        wagner-wst2  wilson  hybrid  liao-you  manne    
% Best Performance            0.00    33.33    33.14   22.28    44.72   
% Solved                     97.33   100.00    98.06   99.67    99.67   
Avg. % gap                    1.15      NaN     0.00    0.00     0.00   
Avg. time                   655.22   110.08   272.29  348.46   184.82   
Std. dev. of time          1438.44   317.33  1007.44  766.78   496.92   
Avg. MP time                654.84   109.70   263.96  342.42   179.16   
Avg. SP time                  0.38     0.38     8.33    6.04     5.66   
Avg. iterations               3.97     4.00    16.00    7.35     7.17   
Std. dev. of iterations       2.87     2.83    75.70   26.09    24.87   

                                                                         \
                                                                          
                          tba      ts2      ts3    wagner-wst2  wilson    
% Best Performance          0.00     5.52     2.75       0.00      6.32   
% Solved                   88.48    93.99    91.00      86.81     92.49   
Avg. % gap                  5.07     2.02     2.43       2.04     10.81   
Avg. time                1997.67  1408.38  1257.42    2156.84   1264.10   
Std. dev. of time        2387.40  2129.32  2216.75    2531.19   2097.83   
Avg. MP time             1993.06  1400.13  1253.02    2153.58   1255.02   
Avg. SP time                4.61     8.25     4.40       3.27      9.07   
Avg. iterations             5.45     6.53     6.20       5.63      6.80   
Std. dev. of iterations     3.46    11.80     7.42       4.81     12.40   

                                                                               \
                          10x4                                                  
                         hybrid  liao-you  manne     tba      ts2      ts3      
% Best Performance         63.46    21.32    46.01     0.00     3.28     0.82   
% Solved                   88.33    96.49    96.33    75.96    86.64    81.50   
Avg. % gap                  0.10     1.52    11.58     3.55     3.73     3.30   
Avg. time                1295.99   938.99   702.98  3083.08  2018.26  2234.93   
Std. dev. of time        2433.50  1649.13  1536.39  2735.12  2492.26  2690.48   
Avg. MP time             1221.26   874.29   632.85  3053.89  1978.69  2185.40   
Avg. SP time               74.73    64.70    70.13    29.19    39.57    49.53   
Avg. iterations            43.60     9.32     8.32     6.29     7.26     7.23   
Std. dev. of iterations    99.80    14.60     8.75     3.81     6.63     6.55   

                                              
                                              
                        wagner-wst2  wilson   
% Best Performance            0.00     14.76  
% Solved                     74.12     85.83  
Avg. % gap                    3.53      3.20  
Avg. time                  3230.96   1838.38  
Std. dev. of time    

In [38]:
pd.set_option('display.max_columns', None)
allowed_keys = [(x, y, z) for (x, y, z) in per_instance_stats.keys() if (x == 'ying' and y in ['10x5', '15x5'])]
per_instance_stats2 = { your_key: per_instance_stats[your_key] for your_key in allowed_keys }
df_table3b = pd.DataFrame.from_dict(per_instance_stats2)
df_table3b

ying                                                 \
                          10x5                                                  
                         hybrid  liao-you  manne     tba      ts2      ts3      
% Best Performance         31.63    14.51    58.84     0.51    13.33     2.85   
% Solved                   98.00    94.32    97.16    64.83    80.13    82.14   
Avg. % gap                  0.02     0.88     7.80     2.15     2.07     2.02   
Avg. time                 585.18  1388.55   746.49  3593.23  2530.59  2253.94   
Std. dev. of time        1448.87  2053.52  1462.37  3008.97  2763.32  2754.93   
Avg. MP time              496.75  1056.01   447.57  3412.20  2276.49  2050.30   
Avg. SP time               88.43   332.53   298.92   181.03   254.09   203.63   
Avg. iterations             9.21     9.84     8.00     7.00     8.59     6.84   
Std. dev. of iterations    26.80    24.28    13.83     6.97    13.58     5.47   

                                                                               \
                                               15x5                             
                        wagner-wst2  wilson   hybrid   liao-you   manne   tba   
% Best Performance            0.00      9.95   100.00     22.22     63.16 NaN   
% Solved                     66.94     73.79    70.00     12.86     14.79 NaN   
Avg. % gap                    2.94      2.08     2.68      5.31      3.93 NaN   
Avg. time                  3729.60   2816.99  6085.90  13544.31  13369.02 NaN   
Std. dev. of time          2902.44   2967.78  6281.25   2959.77   3347.40 NaN   
Avg. MP time               3465.69   2598.16  5560.78  11698.11  11270.32 NaN   
Avg. SP time                263.91    218.84   525.12   1846.20   2098.70 NaN   
Avg. iterations               6.21      8.90     7.60      3.03      3.34 NaN   
Std. dev. of iterations       5.63     13.81     3.73      1.33      1.57 NaN   

                                                             
                                                             
                           ts2    ts3 wagner-wst2   wilson   
% Best Performance           0.00 NaN     NaN          0.00  
% Solved                     8.00 NaN     NaN          6.25  
Avg. % gap                  23.01 NaN     NaN          5.91  
Avg. time                14012.99 NaN     NaN      13731.01  
Std. dev. of time         1439.04 NaN     NaN       2672.68  
Avg. MP time             10928.87 NaN     NaN      13396.19  
Avg. SP time              3084.12 NaN     NaN        334.82  
Avg. iterations              2.16 NaN     NaN          2.90  
Std. dev. of iterations      0.85 NaN     NaN          0.93

In [39]:
# Save output table as HTML
pd.set_option('colheader_justify', 'center')   # FOR TABLE <th>

html_string = '''
<html>
  <head><title>HTML Pandas Dataframe with CSS</title></head>
  <link rel="stylesheet" type="text/css" href="df_style.css"/>
  <body>
    {table}
  </body>
</html>.
'''

# OUTPUT AN HTML FILE
with open(os.path.join(outputfolder_table, 'instance_stats_1.html'), 'w') as f:
    f.write(html_string.format(table=df_table3a.to_html(classes='mystyle')))
with open(os.path.join(outputfolder_table, 'instance_stats_2.html'), 'w') as f:
    f.write(html_string.format(table=df_table3b.to_html(classes='mystyle')))